# PRMS Profiling
Get timing results for PRMS 5.2.1 (fortran) for multiple domains and choice of output.

In [ ]:
# Inspired by test_data/scripts/test_run_domains.py
import os
import pathlib as pl
import shutil
from flopy import run_model

from pywatershed.constants import __pywatershed_root__ as pws_root

repo_root = pws_root.parent
data_dir = pl.Path("../../../data/")

def time_prms_run(domain_dir, exe, working_dir, io, overwrite=False, ):
    
    domain_dir = pl.Path(domain_dir)
    
    working_dir = pl.Path(working_dir)
    if not working_dir.exists():
        working_dir.mkdir(parents=True)
    elif overwrite:
        shutil.rmtree(working_dir)
        working_dir.mkdir(parents=True)
    else:
        raise FileExistsError(f"working_dir exists and overwrite=False: {working_dir}")

    control_dir = data_dir / 'pynhm/performance_runs/PRMS/control_files')
    n_hru_desc = 'multi'
    if domain_dir.name == 'hru_1':
        n_hru_desc = 'single'
    control_file_in = control_dir / f"control.{n_hru_desc}_hru_{io}_io"
    control_file = working_dir / 'control.test'
    shutil.copy2(control_file_in, control_file)   
    print(control_file_in)
    assert control_file.exists()
        
    # the control specifies the output here
    (working_dir / 'output').mkdir()
    
    # Copy the requisite files 
    req_files = ['myparam.param', 'prcp.cbh', 'tmax.cbh', 'tmin.cbh', 'sf_data']
    for ff in req_files:
        shutil.copy2(domain_dir / ff, working_dir / ff)
            
    # print(f"Running '{control_file}' in {working_dir}\n",flush=True,)
    def run_model_time():
        success, buff = run_model(
            exe,
            control_file,
            model_ws=working_dir,
            cargs=[
                "-MAXDATALNLEN",
                "60000",
            ],
            silent=True,
            normal_msg="Normal completion of PRMS",
        )
        assert success, f"could not run prms model in '{working_dir}'"
        return

    
    if "conus" in str(domain_dir): 
        result = %timeit -o -n1 -r1 run_model_nhm()
    else:
        result = %timeit -o run_model_time()

    return result
    #print(f"Ran in {working_dir}\n", flush=True)

In [ ]:
# domain_dir: run_dir/working_dir

prms_run_dir = data_dir / "pywatershed/performance_runs/PRMS"
test_dom_dir = repo_root / 'test_data/'
conus_dom_dir = data_dir / "pynhm/conus_2yr"

dom_dirs = [
    # test_dom_dir / 'hru_1', 
    # test_dom_dir / 'drb_2yr', 
    # test_dom_dir / 'ucb_2yr',
    conus_dom_dir,
]

run_dict = {}
for dom in dom_dirs:
    for io in ['no', 'yes']:
        run_dir = f"{dom.name}_io_{io}"
        src_dir = f"{dom}"
        run_dict[f"{prms_run_dir / run_dir}"] = src_dir

In [ ]:
run_dict

In [ ]:
exe = repo_root / 'bin/prms_mac'
results = {}
for run_dir, src_dir in run_dict.items():
    print(run_dir)
    io = 'yes'
    if run_dir[-2:] == 'no': 
        io = 'no'
    results[run_dir] = time_prms_run(domain_dir=src_dir, exe=exe, working_dir=run_dir, io=io, overwrite=True)

In [ ]:
results

In [ ]:
import pickle
for path, result in results.items():
    path = pl.Path(path)
    pkl_path = path.parent.parent /  (f"results/{path.parent.name}_{path.name}.pkl")
    print(pkl_path)
    
    with open(pkl_path, "wb") as output_file:
        pickle.dump(result, output_file)

In [ ]:
import os; os.getcwd()

In [ ]:
results2 = {}
files = pl.Path('/Users/jamesmcc/usgs/data/pynhm/performance_runs/results/').glob('*.pkl')
for ff in files: 
    print(ff)
    with open(ff, "rb") as input_file:
        results2[ff.name[0:-4]] = pickle.load(input_file)

In [ ]:
results2

In [ ]:
results2['PRMS_drb_2yr_io_yes'].average


In [ ]:
import time
def test():
    time.sleep(4)
    return 10

In [ ]:
test()

In [ ]:
results = %timeit -o (test())